## Итоговый проект по Автоматической обработке естественного языка

#### Таня Гнедина и Настя Шабаева (БКЛ181)

### Описание:

#### 1) Корпус

Корпус состоит из 31 рассказа Виктора Пелевина. Сборник рассказов был найден в интернете в формате txt, поэтому собирать тексты по отдельности не пришлось. Единственное, что пришлось сделать, - вручную проставить в файле теги (<tag></tag>), чтобы отделить рассказы друг от друга. 
Всего в корпусе 107969 слов (28041 уникальное слово). 

Небольшое дополнение: в процессе работы над проектом выяснилось, что один из рассказов автор написал одним предложением (рассказ "Водонапорная башня"). Поэтому, когда заданный запрос находится в этом рассказе, рассказ выводится целиком. Это не очень удобно и наглядно, но это не наша ошибка, а воля автора.

#### 2) Теги разметки

В нашем проекте мы использовали тегсет Mystem, поскольку при анализе слов он ориентируется на их контекст в тексте (это плюс в спорных ситуациях с омонимией), а также потому что Mystem показал достаточно высокий результат в наших вторых дз (на оценку разных морфологических анализаторов).

#### 3) Запросы

Запрос может состоять максимум из 3х слов.

Виды запросов:

'"тамбура"' - функция ищет все предложения, где встретилась данная конкретная форма

'тамбур' (без дополнительных кавычек, любая форма, не только начальная) - функция ищет предложения, в которых встретилась любая форма слова

'дорогой+S' - функция ищет все предложения, в которых встретилась данная словоформа (только она), и при этом словоформа является заданной частью речи (то есть по данному запросу должны находиться предложения, где "дорогой" - форма слова "дорога")

'S' - функция ищет все предложения, в которых встретилась данная часть речи


Кроме того, все перечисленные выше типы запросов могут комбинироваться. То есть возможны запросы: 'подняла S "и"', 'дорогой+A S' и всякие другие комбинации.


### Сам код

#### Читаем файл

In [111]:
with open('rasskazi.txt') as f:
    text = f.read()

#### Импорт

In [112]:
import nltk
import re
from pymystem3 import Mystem
m = Mystem()

#### Предобработка

С помощью регулярных выражений достаем из файла все тексты и их названия
Создаем отдельный список с названиями рассказов и с их текстами

In [113]:
texts = re.findall(r"<tag = '.+?'> (.+?)</tag>", text, re.DOTALL)
names = re.findall(r"<tag = '(.+?)'> .+?</tag>", text, re.DOTALL)

Для примера, начало первого рассказа с его названием

In [114]:
print(names[0], '\n')
print(texts[0][:2002])

Виктор Пелевин. Бубен верхнего мира 

Войдя в тамбур, милиционер мельком глянул на Таню и Машу, перевел взгляд в угол и удивленно уставился на сидящую там женщину.
Женщина и вправду выглядела дико. По ее монголоидному лицу, похожему на загибающийся по краям трехдневный блин из столовой, нельзя было ничего сказать о ее возрасте – тем более что глаза женщины были скрыты кожаными ленточками и бисерными нитями. Несмотря на теплую погоду, на голове у нее была меховая шапка, по которой проходили три широких кожаных полосы – одна охватывала лоб и затылок, и с нее на лицо, плечи и грудь свисали тесемки с привязанными к ним медными человечками, бубенцами и бляшками, а две других скрещивались на макушке, где была укреплена грубо сделанная металлическая птица, задравшая вверх длинную перекрученную шею.
Одета женщина была в широкую самотканую рубаху с тонкими полосами оленьего меха, расшитую кожаной тесьмой, блестящими пластинками и большим количеством маленьких колокольчиков, издававших при каждо

Делим каждый текст на предложения. То есть создаем список текстов, в котором каждому тексту соответствует список его предложений.

In [74]:
list_text_sents = []
for text in texts:
    list_text_sents.append(nltk.sent_tokenize(text))

##### Основная разметка

Все данные будут храниться в словаре (library).

В library ключи - названия рассказов; значения - следующие словари(help_dict).

В help_dict ключи - предложения; значения - также словари (words).
                                       
В words ключи - номера слова в предложении (с нуля); значения - кортежи. В кортеже: само слово, лемма и частеречный тег (в том же порядке).

In [6]:
library = {} 
for i in range(len(names)):   # делаем столько проходов, сколько у нас текстов (каждый проход для одного текста)
    help_dict = {}   
    for sent in list_text_sents[i]:   # проходим по всем предложениям в i-ом тексте
        words = {}   
        word_num = 0 # ключ самого внутреннего словарика
        for ana in m.analyze(sent):   # подаем предложение на анализ и проходим по выданному списку
            if 'analysis' in ana:   # отбраковываем пунктуацию и тд
                if ana['analysis'] == []:   # вот сюда попадают, например, иностранные слова, в разборе у них просто пустой список
                    words[word_num] = (ana['text'].lower())
                    word_num += 1
                else:   # если анализ есть и он нормальный
                    gr = ana['analysis'][0]['gr']
                    pos = gr.split('=')[0].split(',')[0]   #выделяем pos tag
                    lex = ana['analysis'][0]['lex']   # выделяем лексему
                    word = ana['text']   # выделяем само слово
                    words[word_num] = (word.lower(), lex, pos)   #заполняем словарик со всеми словами предложения
                    word_num += 1
        help_dict[sent] = words   #добавляем самый внутренний словарик в средний словарик (который для каждого текста)
    library[names[i]] = help_dict   #добавляем средний словарик в большой со всеми данными

В качестве примера, самый внутренний словарь для первого предложения

In [77]:
print(list(list(library.values())[0].values())[0])

{0: ('войдя', 'входить', 'V'), 1: ('в', 'в', 'PR'), 2: ('тамбур', 'тамбур', 'S'), 3: ('милиционер', 'милиционер', 'S'), 4: ('мельком', 'мельком', 'ADV'), 5: ('глянул', 'глянуть', 'V'), 6: ('на', 'на', 'PR'), 7: ('таню', 'таня', 'S'), 8: ('и', 'и', 'CONJ'), 9: ('машу', 'маша', 'S'), 10: ('перевел', 'переводить', 'V'), 11: ('взгляд', 'взгляд', 'S'), 12: ('в', 'в', 'PR'), 13: ('угол', 'угол', 'S'), 14: ('и', 'и', 'CONJ'), 15: ('удивленно', 'удивленно', 'ADV'), 16: ('уставился', 'уставляться', 'V'), 17: ('на', 'на', 'PR'), 18: ('сидящую', 'сидеть', 'V'), 19: ('там', 'там', 'ADVPRO'), 20: ('женщину', 'женщина', 'S')}


#### Функция поиска

In [59]:
def search_new(request):
    word_1 = '' # первое слово запроса
    word_2 = '' # второе слово запроса
    word_3 = '' # третье слово запроса
    list_request = request.split(' ') # делим запрос на слова по пробелам
    word_1 = list_request[0]   # далее обозначаем слова запроса и выводим сообщение об ошибке, если в запросе больше 3 слов
    if len(list_request) == 2:
        word_2 = list_request[1]
    else:
        if len(list_request) == 3:
            word_2 = list_request[1]
            word_3 = list_request[2]
        else:
            if len(list_request) > 3:
                print("К сожалению, такие запросы обрабатывать я не умею :(")           
    list_reponse = [] # создаем список для ответов
    trigger = 0  # триггер для перехода от первого слова запроса, ко второму
    trigger_2 = 0#ещё один триггер для перехода от второго слова запроса, к третьему
    if len(list_request) <= 3: # если запрос подходящей длины
        for name in library: # проходим по каждому тексту
            for sent in library[name]:# проходим по каждому предложению
                for num in library[name][sent]:# проходим по каждому слову
                    a = library[name][sent] # замена просто для экономии места
                    if (len(a[num]) == 3): # проверка, что есть полный разбор слова
# блок для первого слова, рассматриваем все 4 случая запроса для первого слова; в случае успеха триггеру присваиваем 1
                        if ('"' or "'") in word_1: # если первое слово в кавычках
                            word_clean = word_1[1:-1] # очищаем от кавычек
                            if (a[num][0] == word_clean): # если первый элемент кортежа совпадает с превом словом запроса
                                trigger = 1
                        else:
                            if '+' in word_1: # если в "слове" есть плюс
                                wor,pos = word_1.split('+')  # делим слово запроса по плюсу
                                if (a[num][0] == wor) and (a[num][2] == pos): # сравниваем первую часть запроса с словом в предложении, а вторую с частеречным тегом
                                    trigger = 1
                            else:
                                if not word_1.isupper(): # если слово не полностью из заглавных букв
                                    lemma = m.analyze(word_1)[0]['analysis'][0]['lex'] # определяем лемму запроса
                                    if (a[num][1] == lemma): # сравниваем лемму со вторым элементом кортежа
                                        trigger = 1
                                else:
                                    if (a[num][2] == word_1): # в остальных случаях сравниваем запрос с третьим элементом кортежа
                                        trigger = 1
                    if (len(list_request) == 1) and (trigger == 1): # проверка: если в запросе всего 1 слово и рассматриваемое в данный момент слово удовлетворяет запросу (триггер), то добавляем предложение и название текста в список с ответами
                        list_reponse.append((sent, name))
# второй блок для второго слова, те же 4 случая, в случае успеха присваиваем второму триггеру 1. перед тем как начать этот блок проверяем, что нынешнее слово - раньше предпоследнего (чтобы не выйти за границы словаря), что нынешнее слово удовлетворяет запросу (триггер), что у следующего слова есть разбор и то, что длина запроса больше 1
                    if (num < len(a)-2) and (len(a[num+1]) == 3) and (trigger == 1) and (len(list_request) > 1): 
                        if ('"' or "'") in word_2:
                            word_clean_2 = word_2[1:-1]
                            if (a[num+1][0] == word_clean_2):
                                trigger_2 = 1
                        else:
                            if '+' in word_2:
                                wor,pos = word_2.split('+')
                                if (a[num+1][0] == wor) and (a[num+1][2] == pos):
                                    trigger_2 = 1
                            else:
                                if not word_2.isupper():
                                    lemma = m.analyze(word_2)[0]['analysis'][0]['lex']
                                    if (a[num+1][1] == lemma):
                                        trigger_2 = 1
                                else:
                                    if (a[num+1][2] == word_2):
                                        trigger_2 = 1
                    if (len(list_request) == 2) and (trigger_2 == 1): # если запрос из 2 слов и второй триггер равен 1 (то есть n+1 слово подходит запросу), добавляем предложение и название в список с ответами
                        list_reponse.append((sent, name))
# третий блок для третьего слова, те же 4 случая, в случае успеха запихиваем предложение и название текста в список с ответами. перед тем как начать этот блок проверяем, что нынешнее слово - раньше предпоследнего, что второе слово удовлетворяет запросу (триггер_2), что у n+1 (то есть 3его подряд) слова есть разбор и то, что длина запроса больше 2
                    if (num < len(a)-2) and (len(a[num+2]) == 3) and (trigger_2 == 1) and (len(list_request) > 2):
                        if ('"' or "'") in word_3:
                            word_clean_3 = word_3[1:-1]
                            if (a[num+2][0] == word_clean_3):
                                list_reponse.append((sent, name))
                        else:
                            if '+' in word_3:
                                wor,pos = word_3.split('+')
                                if (a[num+2][0] == wor) and (a[num+2][2] == pos):
                                    list_reponse.append((sent, name))
                            else:
                                if not word_3.isupper():
                                    lemma = m.analyze(word_3)[0]['analysis'][0]['lex']
                                    if (a[num+2][1] == lemma):
                                        list_reponse.append((sent, name))
                                else:
                                    if (a[num+2][2] == word_3):
                                        list_reponse.append((sent, name))
                    trigger = 0 #обнуляем триггеры перед следующим проходом 
                    trigger_2 = 0
    for item in list_reponse: # проходим по списку и печатаем удачные предложения
        print(item[0], ' (', item[1], ')\n')

#### Примеры запросов:

In [87]:
search_new('кожаного')

По ее монголоидному лицу, похожему на загибающийся по краям трехдневный блин из столовой, нельзя было ничего сказать о ее возрасте – тем более что глаза женщины были скрыты кожаными ленточками и бисерными нитями.  ( Виктор Пелевин. Бубен верхнего мира )

Несмотря на теплую погоду, на голове у нее была меховая шапка, по которой проходили три широких кожаных полосы – одна охватывала лоб и затылок, и с нее на лицо, плечи и грудь свисали тесемки с привязанными к ним медными человечками, бубенцами и бляшками, а две других скрещивались на макушке, где была укреплена грубо сделанная металлическая птица, задравшая вверх длинную перекрученную шею.  ( Виктор Пелевин. Бубен верхнего мира )

Одета женщина была в широкую самотканую рубаху с тонкими полосами оленьего меха, расшитую кожаной тесьмой, блестящими пластинками и большим количеством маленьких колокольчиков, издававших при каждом толчке вагона довольно приятный мелодичный звон.  ( Виктор Пелевин. Бубен верхнего мира )

В руках женщина держа

In [88]:
search_new('дорогой+S')

Он снизился, сбавил скорость и описал над головой одуревшей женщины несколько кругов, потом поднялся вверх, застыл на месте и строго, как флюгер, повернул над дорогой.  ( Виктор Пелевин. Жизнь и приключения сарая номер XII. )

Девушка, дремавшая возле гриля, встала, подошла к стойке и со знакомой ненавистью глянула на люсину лисью шубу («пятнадцать кусков», как её называли подруги), лисью шапку и на чуть тронутое дорогой косметикой лицо, глядевшее на неё из заснеженного тёмного мира.  ( Виктор Пелевин. Миттельшпиль )

А уж дальше, за Окружной дорогой, вообще непонятно что начинается.  ( Виктор Пелевин. СССР Тайшоу Чжуань )

Зарплата на самом деле охуеннная, просто такой дорогой кокаин.  ( Виктор Пелевин. Тайм-аут, или Вечерняя Москва )



In [90]:
search_new('дорогой+A')

Грустно было Маше глядеть на это, и собственная жизнь, начатая двадцать пять лет назад неведомой волей, вдруг показалась ей такой же точно дорогой – сначала прямой и ровной, обсаженной ровными рядами простых истин, а потом забытой неизвестным начальством и превратившейся в непонятно куда ведущую кривую тропу.  ( Виктор Пелевин. Бубен верхнего мира )

Лучше всего, конечно, – испанец из «Голубой дивизии», но это дорогой покойник.  ( Виктор Пелевин. Бубен верхнего мира )

Водонапорная башня вполне может оказаться тем первым, с чего начнется все остальное, потому что предметы появляются тогда, когда становятся известны их названия, и происходящее за окном сразу приобретает смысл – солдаты заканчивают работу, выкладывая белым кирпичом число «1928» на толстой верхней части каменного цилиндра, и даже не догадываются, что кто-то следит за тем, что они делают, думая об этом почти без помощи слов, но очень серьезно: любая башня или даже труба сначала строится таким образом, будто должна поднятьс

In [89]:
search_new('"тамбура"')

Милиционер с сомнением посмотрел на ту, кого Таня назвала старухой – она тихо сидела в углу тамбура, покачиваясь вместе с вагоном, и не обращала никакого внимания на скандал по ее поводу.  ( Виктор Пелевин. Бубен верхнего мира )

Маша глядела в окно тамбура на проносящиеся мимо огороды и домики и тихо плакала.  ( Виктор Пелевин. Бубен верхнего мира )



In [115]:
search_new('"какой-то" S')

И тут в какой-то палатке ему водку не продали.  ( Виктор Пелевин. Бубен верхнего мира )

Но интересно вот что: когда поднимаешься из долины к безлюдным заснеженным пикам, пересекаешь много природных зон и в какой-то момент замечаешь, что прямо у обочины шоссе начинается березовая роща, дальше растут рябины и липы, и кажется, что вот-вот в просвете между деревьями покажутся скромные домики обычного русского села, пара коров, пасущихся за околицей, и, конечно же, маковка маленькой бревенчатой церкви.  ( Виктор Пелевин. Вести из Непала )

Водонапорная башня вполне может оказаться тем первым, с чего начнется все остальное, потому что предметы появляются тогда, когда становятся известны их названия, и происходящее за окном сразу приобретает смысл – солдаты заканчивают работу, выкладывая белым кирпичом число «1928» на толстой верхней части каменного цилиндра, и даже не догадываются, что кто-то следит за тем, что они делают, думая об этом почти без помощи слов, но очень серьезно: любая башня 

In [116]:
search_new('какая-то S')

Обычно возле железнодорожных станций бывают хоть какие-то поселения людей, а здесь не было ничего, кроме кирпичной избушки кассы, и увязать это место можно было только с расстоянием до Москвы.  ( Виктор Пелевин. Бубен верхнего мира )

И тут в какой-то палатке ему водку не продали.  ( Виктор Пелевин. Бубен верхнего мира )

Расстегнув шубку (под ней была белая кофточка с широкой черной полосой на груди) и прижавшись к зеркалу, чтобы пропустить двух работяг в ватниках, горячо обсуждавших на ходу какое-то дело (и так махавших при этом руками, что не дай Бог кому-нибудь было оказаться на пути огромных растрескавшихся кулаков), она увидела почти вплотную свое припудренное лицо с ясно заметными морщинками у глаз.  ( Виктор Пелевин. Вести из Непала )

На мешках были оттиснуты какие-то цифры, черные зонтики и надписи «КРЮЧЬЯМИ НЕ ПОЛЬЗОВАТЬСЯ».  ( Виктор Пелевин. Вести из Непала )

Но интересно вот что: когда поднимаешься из долины к безлюдным заснеженным пикам, пересекаешь много природных зон 

In [92]:
search_new('русский села+S')

Но интересно вот что: когда поднимаешься из долины к безлюдным заснеженным пикам, пересекаешь много природных зон и в какой-то момент замечаешь, что прямо у обочины шоссе начинается березовая роща, дальше растут рябины и липы, и кажется, что вот-вот в просвете между деревьями покажутся скромные домики обычного русского села, пара коров, пасущихся за околицей, и, конечно же, маковка маленькой бревенчатой церкви.  ( Виктор Пелевин. Вести из Непала )



In [102]:
search_new('SPRO села+V')

Тыймы вниз не пошла – она села на Танину сумку, прислонилась спиной к дереву и замерла.  ( Виктор Пелевин. Бубен верхнего мира )



In [97]:
search_new('рядом+S')

Он мог, например, видеть сон, где действие происходило в женской бане (довольно частое и странное видение, поражавшее целым рядом нелепостей: на бревенчатых стенах висели рукописные плакаты со стихами, призывавшими беречь хлеб, а кряжистые русоволосые бабы со ржавыми шайками в руках носили короткие балетные юбочки из перьев), – и одновременно с этим мог не только следить за потеком яичного желтка на лекторском галстуке, но и выслушивать анекдот про трех грузин в космосе, который постоянно рассказывал сосед.  ( Виктор Пелевин. Спи )



In [99]:
search_new('сесть рядом+ADV')

– Да, – ответила Любочка, садясь рядом, – прочла.  ( Виктор Пелевин. Вести из Непала )



In [103]:
search_new('сесть PR S')

Маша села на землю рядом с подругой.  ( Виктор Пелевин. Бубен верхнего мира )

Колемасов сел за стол и потянул к себе фишки.  ( Виктор Пелевин. Вести из Непала )

Водонапорная башня вполне может оказаться тем первым, с чего начнется все остальное, потому что предметы появляются тогда, когда становятся известны их названия, и происходящее за окном сразу приобретает смысл – солдаты заканчивают работу, выкладывая белым кирпичом число «1928» на толстой верхней части каменного цилиндра, и даже не догадываются, что кто-то следит за тем, что они делают, думая об этом почти без помощи слов, но очень серьезно: любая башня или даже труба сначала строится таким образом, будто должна подняться до самого неба, чем обязательно завершилось бы простое добавление новых кирпичных колец изо дня в день, если бы не решение строителей уйти, приводящее к тому, что какой-то кирпич обязательно становится последним, а я – единственным свидетелем остановки работ, потому что во всем доме напротив только я понимаю

In [106]:
search_new('"и" "вдруг" V')

Словно пытаясь что-то вспомнить, она огляделась по сторонам и вдруг увидела в нескольких шагах от себя того самого человека в гладиаторском шлеме.  ( Виктор Пелевин. Греческий вариант )

Сначала он не мог взять в толк, что это и зачем, – и вдруг вспомнил: ведь столько было когда-то связано с этой штукой!  ( Виктор Пелевин. Жизнь и приключения сарая номер XII. )

Постепенно окружающие предметы потеряли свою бесчеловечность, мир как-то разгладился, и вдруг произошла совершенно неожиданная вещь.  ( Виктор Пелевин. Музыка со столба )

И вдруг заметил, что это непонятно что и есть он сам и это оно смотрит на все остальное, только что считавшее себя им, и пытается разобраться в том, что только что пыталось разобраться в нем самом.  ( Виктор Пелевин. Музыка со столба )

Достав варган из кармана шубы, он уже поднес его ко рту, и вдруг понял, на что тот был похож.  ( Виктор Пелевин. Нижняя тундра )

Юань Мэн закрыл глаза и вдруг вспомнил странную картинку – он, кажется, видел ее в каком-то журн

In [110]:
search_new('"и" "вдруг" S')

И вдруг голубь сел на капот.  ( Виктор Пелевин. Нижняя тундра )

Их взгляды встретились, Дарвин поднял руку, чтобы положить ее собеседнику на плечо, и вдруг глаза капитана словно выцвели – восхищенное внимание в них сменилось почти физически ощутимым страхом.  ( Виктор Пелевин. Происхождение видов )



In [107]:
search_new('её+SPRO')

Девушка, дремавшая возле гриля, встала, подошла к стойке и со знакомой ненавистью глянула на люсину лисью шубу («пятнадцать кусков», как её называли подруги), лисью шапку и на чуть тронутое дорогой косметикой лицо, глядевшее на неё из заснеженного тёмного мира.  ( Виктор Пелевин. Миттельшпиль )

Перед каждым желтело по нетронутому стакану с коктейлем, а на полу под столиком стояла бутылка водки – они пили через длинную пластиковую трубочку, передавая её друг другу таким же спокойным и точным движением, каким, наверно, нажимали кнопки и переключали тумблеры на пультах своего подводного ракетоносца.  ( Виктор Пелевин. Миттельшпиль )

– именно её всё лето крутил старенький катушечный «Маяк» в штабе стройотряда.  ( Виктор Пелевин. Миттельшпиль )

Всю ночь её из руки не выпускал, подушку пополам разрубил.  ( Виктор Пелевин. Миттельшпиль )

Здесь её часто посещали мысли о бренности существования – а тут ещё вспомнилась знакомая, Наташа, которая нашла себе в мужья пожилого негра и уже совсем 

In [108]:
search_new('её+APRO')

Люся задержалась на секунду у мраморного ограждения, чтобы поглядеть на расписной потолок – в его центре была огромная фреска, изображавшая, как Люся смутно догадывалась, сотворение мира, в котором она родилась и выросла и который за последние несколько лет уже успел куда-то исчезнуть: в центре огромными букетами расплывались огни салюта, а по углам стояли титаны – не то лыжники в тренировочных, не то студенты с тетрадями под мышкой, – Люся никогда не разглядывала их, потому что всё её внимание притягивали стрелы и звёзды салюта, нарисованные какими-то давно забытыми цветами, теми самыми, которыми утро красит ещё иногда стены старого Кремля: сиреневыми, розовыми и нежно-лиловыми, напоминающими о давно канувших в Лету жестяных карамельных коробках, зубном порошке и ветхих настенных календариках, оставшихся вместе с пачкой облигаций от забытой уже бабушки.  ( Виктор Пелевин. Миттельшпиль )

Видно, что-то произошло с её лицом – старший официант посмотрел на неё и сразу отвёл удивлённый вз

In [109]:
search_new('"гнида" сказал S')

– Держи… Райкомовские… Не знаю даже, как к тебе обращаться…
– А как раньше – Гнида, – сказала Люся, и на её глазах блеснули слёзы.  ( Виктор Пелевин. Миттельшпиль )



In [118]:
search_new('ADV') # это последний пример, дальше можно не смотреть, просто у него адски большой вывод, дело усугубляет ещё то, что предложение выводится столько раз, сколько наречий в нем встречается (мы сознательно это не меняли, потому что ошибки в этом не видим)

Войдя в тамбур, милиционер мельком глянул на Таню и Машу, перевел взгляд в угол и удивленно уставился на сидящую там женщину.  ( Виктор Пелевин. Бубен верхнего мира )

Войдя в тамбур, милиционер мельком глянул на Таню и Машу, перевел взгляд в угол и удивленно уставился на сидящую там женщину.  ( Виктор Пелевин. Бубен верхнего мира )

Женщина и вправду выглядела дико.  ( Виктор Пелевин. Бубен верхнего мира )

Женщина и вправду выглядела дико.  ( Виктор Пелевин. Бубен верхнего мира )

По ее монголоидному лицу, похожему на загибающийся по краям трехдневный блин из столовой, нельзя было ничего сказать о ее возрасте – тем более что глаза женщины были скрыты кожаными ленточками и бисерными нитями.  ( Виктор Пелевин. Бубен верхнего мира )

По ее монголоидному лицу, похожему на загибающийся по краям трехдневный блин из столовой, нельзя было ничего сказать о ее возрасте – тем более что глаза женщины были скрыты кожаными ленточками и бисерными нитями.  ( Виктор Пелевин. Бубен верхнего мира )

Не

Это делало все происходящее крайне двусмысленным, потому что если опаздывала Любочка – это было одно, а если инженер по рационализации Любовь Григорьевна Сухоручко – уже совсем другое.  ( Виктор Пелевин. Вести из Непала )

Это делало все происходящее крайне двусмысленным, потому что если опаздывала Любочка – это было одно, а если инженер по рационализации Любовь Григорьевна Сухоручко – уже совсем другое.  ( Виктор Пелевин. Вести из Непала )

– Ничего.  ( Виктор Пелевин. Вести из Непала )

– Нисколько, – ответила Любочка, а потом наморщилась и сказала: – Хотя нет.  ( Виктор Пелевин. Вести из Непала )

Я еще не оформила.  ( Виктор Пелевин. Вести из Непала )

– Понятно.  ( Виктор Пелевин. Вести из Непала )

Уже выплатили.  ( Виктор Пелевин. Вести из Непала )

Ну а мы вам сколько платим?  ( Виктор Пелевин. Вести из Непала )

Итого – сто пятьдесят рублей разницы.  ( Виктор Пелевин. Вести из Непала )

И не только эту мысль, но и многое другое, чего директор, наверное, вовсе не имел в виду.  

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



 )

И потом, разве может что-нибудь скомпрометировать всех этих бедняг?  ( Виктор Пелевин. Греческий вариант )

Да еще в их собственных глазах?  ( Виктор Пелевин. Греческий вариант )

Скорее всего пакет с компроматом, так нигде и не вынырнувший, был легендой, но врагов у Кудрявцева было более чем достаточно, и он мог ожидать удара с любой стороны.  ( Виктор Пелевин. Греческий вариант )

Скорее всего пакет с компроматом, так нигде и не вынырнувший, был легендой, но врагов у Кудрявцева было более чем достаточно, и он мог ожидать удара с любой стороны.  ( Виктор Пелевин. Греческий вариант )

Скорее всего пакет с компроматом, так нигде и не вынырнувший, был легендой, но врагов у Кудрявцева было более чем достаточно, и он мог ожидать удара с любой стороны.  ( Виктор Пелевин. Греческий вариант )

Пошатнувшиеся дела вынудили его резко пересмотреть свой создавшийся в обществе имидж – особенно в связи с тем, что группировка, под контролем которой он действовал, предъявила ему своего рода ультим

При ремонте было найдено несколько человеческих черепов и планшет с секретными документами – но этого Вера не увидела, потому что за ними приехали откуда надо и куда надо увезли.  ( Виктор Пелевин. Девятый сон Веры Павловны )

Работы стало намного меньше, а денег – просто уйма.  ( Виктор Пелевин. Девятый сон Веры Павловны )

Работы стало намного меньше, а денег – просто уйма.  ( Виктор Пелевин. Девятый сон Веры Павловны )

Теперь Вера ходила по помещениям в новом синем халате, вежливо раздвигала толпящихся посетителей и протирала сухой фланелевой тряпочкой стекла прилавков, за которыми новогодней разноцветной фольгой («все мысли веков!  ( Виктор Пелевин. Девятый сон Веры Павловны )

Теперь Вера ходила по помещениям в новом синем халате, вежливо раздвигала толпящихся посетителей и протирала сухой фланелевой тряпочкой стекла прилавков, за которыми новогодней разноцветной фольгой («все мысли веков!  ( Виктор Пелевин. Девятый сон Веры Павловны )

все миры!» – тихонько шептала Вера) мерцали

...

Свита подтащила Вована к двери, над которой висела табличка «ЗАО РАЙ» (Вован принял это как должное, недаром ведь он носил тяжёлую цепь с гимнастом) и втолкнула внутрь.  ( Виктор Пелевин. Тайм-аут, или Вечерняя Москва )

Свита подтащила Вована к двери, над которой висела табличка «ЗАО РАЙ» (Вован принял это как должное, недаром ведь он носил тяжёлую цепь с гимнастом) и втолкнула внутрь.  ( Виктор Пелевин. Тайм-аут, или Вечерняя Москва )

В ней стояла большая бронзовая сковородка, при одном взгляде на которую становилось ясно, что вещь эта невероятно древняя.  ( Виктор Пелевин. Тайм-аут, или Вечерняя Москва )

В ней стояла большая бронзовая сковородка, при одном взгляде на которую становилось ясно, что вещь эта невероятно древняя.  ( Виктор Пелевин. Тайм-аут, или Вечерняя Москва )

На стене над ней висел такой же древний бронзовый термометр непонятного принципа, у него внутри зеленела какая-то спираль, на циферблате, к которому подходила грубая стрелка, была только одна отметина.  ( Вик

Красиво.  ( Виктор Пелевин. Тарзанка )

Но главное все же не в этом.  ( Виктор Пелевин. Тарзанка )

Иногда мне кажется, что есть часть души, которая все время спит и только летней ночью на несколько секунд просыпается, чтобы выглянуть наружу и что-то такое вспомнить, как оно было – давно и не здесь… Синева… Звезды… Тайна…
4
Вскоре идти стало заметно труднее.  ( Виктор Пелевин. Тарзанка )

Иногда мне кажется, что есть часть души, которая все время спит и только летней ночью на несколько секунд просыпается, чтобы выглянуть наружу и что-то такое вспомнить, как оно было – давно и не здесь… Синева… Звезды… Тайна…
4
Вскоре идти стало заметно труднее.  ( Виктор Пелевин. Тарзанка )

Иногда мне кажется, что есть часть души, которая все время спит и только летней ночью на несколько секунд просыпается, чтобы выглянуть наружу и что-то такое вспомнить, как оно было – давно и не здесь… Синева… Звезды… Тайна…
4
Вскоре идти стало заметно труднее.  ( Виктор Пелевин. Тарзанка )

Иногда мне кажется, что 

Что-то похожее на вкус во рту явственно ощущалось и в душе и было, кажется, связано со вчерашним разговором, хотя его содержание, тема и даже примерная траектория совершенно не желали вспоминаться.  ( Виктор Пелевин. Ухтяб )

Словно бы что-то застряло в мозгу, обособившись от всего остального, и теперь ощущалось, как плотная масса посреди знакомых мыслей – холодная, бесформенная и угрожающая.  ( Виктор Пелевин. Ухтяб )

«Вспомнить надо, – думал Маралов, – о чем-то мы таком… О часах, что ли?  ( Виктор Пелевин. Ухтяб )

Час, наверно, бредил… И вот тогда я чего-то такое… Нет, не помню».  ( Виктор Пелевин. Ухтяб )

Так продолжалось довольно долго.  ( Виктор Пелевин. Ухтяб )

Так продолжалось довольно долго.  ( Виктор Пелевин. Ухтяб )

Маралов не то чтоб не мог вспомнить, в чем было дело, а скорее не мог себя заставить сделать это, как никогда не мог себя заставить сразу нырнуть в холодную воду.  ( Виктор Пелевин. Ухтяб )

Маралов не то чтоб не мог вспомнить, в чем было дело, а скорее не мо

Вы… Да я вас в газетах… В «Новом В'гемени»… – затараторил он, причем стало сразу ясно, что если он и имеет какое-то отношение к газетам, то уж во всяком случае не к «Новому Времени», – наглость какая… Да вы знаете, с кем гово'гите?  ( Виктор Пелевин. Хрустальный мир )

Было какое-то несоответствие между его возмущенным тоном и готовностью, с которой он начал пятиться из пятна света назад, в темноту, – слова предполагали, что сейчас начнется долгий и тяжелый скандал, а движения показывали немедленную готовность даже не убежать, а именно задать стрекача.  ( Виктор Пелевин. Хрустальный мир )

Было какое-то несоответствие между его возмущенным тоном и готовностью, с которой он начал пятиться из пятна света назад, в темноту, – слова предполагали, что сейчас начнется долгий и тяжелый скандал, а движения показывали немедленную готовность даже не убежать, а именно задать стрекача.  ( Виктор Пелевин. Хрустальный мир )

– В городе чрезвычайное положение, – закричал ему вслед Николай, – подышите 


Начать обязательно надо было с того, чтобы встать в пять тридцать утра и облиться холодной водой, а дальше была такая уйма вариантов, что остановиться на чем-нибудь конкретном было крайне тяжело, и Николай стал напряженно выбирать, незаметно для себя приборматывая вслух и сжимая от возбуждения кулаки.  ( Виктор Пелевин. Хрустальный мир )

Начать обязательно надо было с того, чтобы встать в пять тридцать утра и облиться холодной водой, а дальше была такая уйма вариантов, что остановиться на чем-нибудь конкретном было крайне тяжело, и Николай стал напряженно выбирать, незаметно для себя приборматывая вслух и сжимая от возбуждения кулаки.  ( Виктор Пелевин. Хрустальный мир )

Начать обязательно надо было с того, чтобы встать в пять тридцать утра и облиться холодной водой, а дальше была такая уйма вариантов, что остановиться на чем-нибудь конкретном было крайне тяжело, и Николай стал напряженно выбирать, незаметно для себя приборматывая вслух и сжимая от возбуждения кулаки.  ( Виктор Пеле